In [14]:
import pandas as pd

In [15]:
estacion = "NORESTE2"

d20_21 = pd.read_excel('DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx', sheet_name=estacion)
d22_23 = pd.read_excel('DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx', sheet_name=estacion)
df = pd.concat([d20_21,d22_23])

In [16]:
df['date'] = pd.to_datetime(df['date'])
df["deltaDate"] = (df["date"] - df.iloc[0,0]).apply(lambda x: x.value/3.6e+12)
df.head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,deltaDate
0,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,94.0,22.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
1,2020-01-01 01:00:00,1.29,NaN,NaN,NaN,NaN,19.0,NaN,718.4,0.02,94.0,NaN,0.0,11.38,1.8,NaN,1.0
2,2020-01-01 02:00:00,1.29,NaN,NaN,NaN,NaN,17.0,NaN,717.2,0.01,95.0,NaN,0.0,11.34,8.7,NaN,2.0
3,2020-01-01 03:00:00,1.25,NaN,NaN,NaN,NaN,25.0,9.0,716.7,0.00,95.0,NaN,0.0,11.39,6.3,NaN,3.0
4,2020-01-01 04:00:00,1.24,NaN,NaN,NaN,NaN,14.0,NaN,716.2,0.01,95.0,NaN,0.0,11.47,5.3,NaN,4.0


In [17]:
print(df.shape)
df.isna().sum()

(31790, 17)


date             0
CO            4291
NO           10636
NO2          10653
NOX          10228
O3           10292
PM10          1762
PM2.5         4808
PRS            889
RAINF          818
RH            1437
SO2           5379
SR             284
TOUT           841
WSR           6344
WDR           4178
deltaDate        0
dtype: int64

In [18]:
from sklearn.impute import KNNImputer as imputer
k = imputer(n_neighbors=4)
datos = k.fit_transform((df.iloc[:, 1:]).to_numpy())

In [19]:
limpios = pd.DataFrame(datos, columns=df.columns[1:])
limpios["date"] = list(df["date"])
limpios.to_csv(f"datos{estacion}.csv")

In [20]:
intervalos={
    "SO2":[0.008, 0.11, 0.165, 0.22],
    "NO2":[8.75, 11, 13.3, 15.5],
    "etiqueta": ["bueno", "aceptable", "mala", "muy mala"]
}
def change(val, col):
    if val< intervalos[col][0]:
        return "bueno"
    if val< intervalos[col][1]:
        return "aceptable"
    if val< intervalos[col][2]:
        return "mala"
    if val< intervalos[col][3]:
        return "muy mala"
    return "Extremadamente mala"

In [21]:
datos = limpios[["date", "deltaDate","NO","NO2", "NOX", "SO2", "TOUT", "RH", "SR", "PM2.5"]]
cols_ppm = ['NO','NO2','NOX','SO2']
for col in cols_ppm:
    datos[col+'PPM'] = datos[col]*0.001
datos["SO2_discret"] = datos["SO2PPM"].apply(lambda x: change(x, "SO2"))
datos.to_csv(f"datos{estacion}.csv")

C:\Users\engel\AppData\Local\Temp\ipykernel_3512\258482312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos[col+'PPM'] = datos[col]*0.001
C:\Users\engel\AppData\Local\Temp\ipykernel_3512\258482312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos[col+'PPM'] = datos[col]*0.001
C:\Users\engel\AppData\Local\Temp\ipykernel_3512\258482312.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [22]:
datos.head()

,date,deltaDate,NO,NO2,NOX,SO2,TOUT,RH,SR,PM2.5,NOPPM,NO2PPM,NOXPPM,SO2PPM,SO2_discret
0,2020-01-01 00:00:00,0.0,27.025,1.625,0.5,54.875,21.12,58.5,0.0,22.00,0.027025,0.001625,0.0005,0.054875,aceptable
1,2020-01-01 01:00:00,1.0,25.525,1.625,0.5,13.225,11.38,94.0,0.0,9.25,0.025525,0.001625,0.0005,0.013225,aceptable
2,2020-01-01 02:00:00,2.0,25.525,1.625,0.5,13.225,11.34,95.0,0.0,8.75,0.025525,0.001625,0.0005,0.013225,aceptable
3,2020-01-01 03:00:00,3.0,25.525,1.625,0.5,13.225,11.39,95.0,0.0,9.00,0.025525,0.001625,0.0005,0.013225,aceptable
4,2020-01-01 04:00:00,4.0,23.525,1.625,0.5,13.550,11.47,95.0,0.0,8.75,0.023525,0.001625,0.0005,0.013550,aceptable


In [23]:
datos["SO2_discret"] = datos["SO2PPM"].apply(lambda x: change(x, "SO2"))

#limpios["NO2_discret"] = limpios["NO2"].apply(lambda x: change(x, "NO2"))
#limpios["SO2_discret"] = (limpios["SO2"]*0.001).apply(lambda x: change(x, "SO2"))

C:\Users\engel\AppData\Local\Temp\ipykernel_3512\3441339858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos["SO2_discret"] = datos["SO2PPM"].apply(lambda x: change(x, "SO2"))


In [24]:
#limpios["SO2_pm"] = limpios["SO2"]*0.001

In [25]:
#limpios.head()

In [26]:
datos = limpios[["date", "deltaDate","NO","NO2", "NOX", "SO2", "TOUT", "RH", "SR", "NO2_discret",	"SO2_discret",	"SO2_pm"]]

KeyError: "['NO2_discret', 'SO2_discret', 'SO2_pm'] not in index"

In [ ]:
datos.head()

,date,deltaDate,NO,NO2,NOX,SO2,TOUT,RH,SR,NO2_discret,SO2_discret,SO2_pm
0,2020-01-01 00:00:00,0.0,4.3,6.375,10.625,0.850,12.2675,92.75,0.000,bueno,bueno,0.000850
1,2020-01-01 01:00:00,1.0,2.0,5.200,7.200,1.775,11.6800,93.00,0.037,bueno,bueno,0.001775
2,2020-01-01 02:00:00,2.0,2.6,5.000,7.600,0.750,11.6800,93.00,0.037,bueno,bueno,0.000750
3,2020-01-01 03:00:00,3.0,3.1,5.900,8.900,0.725,11.7000,93.00,0.037,bueno,bueno,0.000725
4,2020-01-01 04:00:00,4.0,2.0,4.500,6.500,0.775,11.8000,93.00,0.037,bueno,bueno,0.000775


In [ ]:
#datos.to_csv("limpios.csv")